# Machine Translation Project (English to Spanish)

In [1]:
import pathlib
import random
import string
import tensorflow.strings as tf_strings
import tensorflow.data as tf_data
import re
from keras.layers import TextVectorization
import keras
import tensorflow as tf
from keras import layers
import json

In [2]:
text_file =  keras.utils.get_file(
    fname = "spa-eng.zip",
    origin = "http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract = True,
)

text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

with open(text_file, "r") as f:
    lines = f.read().split("\n")[:-1]
    
text_pairs = []

for line in lines:
    eng, spa = line.split("\t")
    spa = "[start] " + spa + " [end]"
    text_pairs.append((eng, spa))

In [3]:
random.shuffle(text_pairs)

In [4]:
for i in range(5):
    print(text_pairs[i])

('Read it once more.', '[start] Léalo otra vez. [end]')
('What are you trying to hide?', '[start] ¿Qué intentáis ocultar? [end]')
('I am never free on Sundays.', '[start] Nunca estoy libre los domingos. [end]')
('A bad cold caused the singer to lose his voice.', '[start] Un serio resfriado le hizo al cantante perder su voz. [end]')
('Do you mind my turning on the radio?', '[start] ¿Te importa que encienda la radio? [end]')


Structure of the Dataset

In [5]:
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

118964 total pairs
83276 training pairs
17844 validation pairs
17844 test pairs


In [6]:
# parameters
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64

## Vectorize the data

In [7]:
def custom_standardization(input_string):
    lowercase = tf_strings.lower(input_string)
    return tf_strings.regex_replace(lowercase, f"[{re.escape(strip_chars)}]", "")

# vectorization
eng_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length,
)

spa_vectorization = TextVectorization(
    max_tokens = vocab_size,
    output_mode = "int",
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization,
)

train_eng_texts = [pair[0] for pair in train_pairs]
train_spa_texts = [pair[1] for pair in train_pairs]

eng_vectorization.adapt(train_eng_texts)
spa_vectorization.adapt(train_spa_texts)

#save the vectorization layers
eng_vectorization_config = eng_vectorization.get_config()
eng_vectorization_config.pop('standardize', None)
eng_vocab = eng_vectorization.get_vocabulary()
with open('eng_vectorization_config.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vectorization_config, f)
    
with open('eng_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(eng_vocab, f)
    
spa_vectorization_config = spa_vectorization.get_config()
spa_vectorization_config.pop('standardize', None)
spa_vocab = spa_vectorization.get_vocabulary()
with open('spa_vectorization_config.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vectorization_config, f)
    
with open('spa_vocab.json', 'w', encoding='utf-8') as f:
    json.dump(spa_vocab, f)
    

def format_dataset(eng, spa):
    eng = eng_vectorization(eng)
    spa = spa_vectorization(spa)
    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": spa[:, :-1],
        },
        spa[:, 1:],
    )
    
def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)
    

In [8]:
for inputs,targets in train_ds.take(1):
    print(inputs["encoder_inputs"].shape)
    print(targets.shape)

(64, 20)
(64, 20)


In [9]:
# Creating an Encoder
@tf.keras.utils.register_keras_serializable()
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True
        
    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
        else:
            padding_mask = None
            
        attention_output = self.attention(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = padding_mask,
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "dense_dim": self.dense_dim,
            "num_heads": self.num_heads,
        })
        return config
    
# Creating a Positional Embedding
@tf.keras.utils.register_keras_serializable()
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim = vocab_size, output_dim = embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim = sequence_length, output_dim = embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start = 0, limit = length, delta = 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
    
    def compute_mask(self, inputs, mask=None):
        if mask is not None:
            return tf.not_equal(inputs, 0)
        else:
            return None
        
    def get_config(self):
        config = super().get_config()
        config.update({
            "vocab_size": self.vocab_size,
            "sequence_length": self.sequence_length,
            "embed_dim": self.embed_dim,
        })
        return config
    
# Creating a Decoder
@tf.keras.utils.register_keras_serializable()
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads = num_heads, key_dim = embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation = "relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True
        
    def call(self, inputs, encoder_outputs, mask=None):
        casual_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype = tf.int32)
            padding_mask = tf.minimum(padding_mask, casual_mask)
        else:
            padding_mask = None
            
        attention_output_1 = self.attention_1(
            query = inputs,
            value = inputs,
            key = inputs,
            attention_mask = casual_mask,
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)
        
        attention_output_2 = self.attention_2(
            query = out_1,
            value = encoder_outputs,
            key = encoder_outputs,
            attention_mask = padding_mask,
        )
        
        out_2 = self.layernorm_2(out_1 + attention_output_2)
        proj_output = self.dense_proj(out_2)
        
        return self.layernorm_3(out_2 + proj_output)
    
    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, tf.int32)
        mask = tf.reshape(mask,(1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [
                tf.expand_dims(batch_size, -1),
                tf.convert_to_tensor([1, 1]),
            ],
            axis = 0,
        )
        return tf.tile(mask, mult)
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "latent_dim": self.latent_dim,
            "num_heads": self.num_heads,
        })
        return config


In [10]:
# define emmbedding dimensions, latent dimensions, and number of heads
embed_dim = 256
latent_dim = 2048
num_heads = 8

#Encoder
encoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "encoder_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)

encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)

encoder = keras.Model(encoder_inputs, encoder_outputs, name = "encoder")

#Decoder
decoder_inputs = keras.Input(shape = (None,), dtype = "int64", name = "decoder_inputs")
encoder_seq_inputs = keras.Input(shape = (None, embed_dim), name = "encoder_seq_inputs")

x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)

x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoder_seq_inputs)

x = layers.Dropout(0.5)(x)

decoder_outputs = layers.Dense(vocab_size, activation = "softmax")(x)

decoder = keras.Model([decoder_inputs, encoder_seq_inputs], decoder_outputs, name = "decoder")

# Define the final model
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name = "transformer"
)


In [11]:
epochs = 20

transformer.summary()

transformer.compile(
    "rmsprop", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"]
)

transformer.fit(train_ds, epochs = epochs, validation_data = val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, None, 256)   3155456     ['positional_embedding[

In [12]:
transformer.save("transformer_model.keras")

In [13]:
spa_vocab = spa_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = sequence_length

def decode_sentence(input_sentence):
    tokenized_input_sentence = eng_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = spa_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = tf.argmax(predictions[0, i, :]).numpy().item(0)
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    input_sentence = input_sentence.lower()
    input_sentence = input_sentence.translate(str.maketrans('', '', strip_chars))
    translated = decode_sentence(input_sentence)
    print(f"input: {input_sentence}")
    print(f"translated: {translated}")
    print()

input: we ate sandwiches for breakfast
translated: [start] me lo [UNK] para desayuno [end]

input: tom is more intelligent than i am
translated: [start] tom es más inteligente que yo [end]

input: she sometimes stays up late at night
translated: [start] a veces se se le queda tarde en la noche [end]

input: tell me something that i dont know
translated: [start] dime algo que yo no sé [end]

input: your name is definitely not on the list
translated: [start] tu nombre es [UNK] en la lista [end]



In [14]:
import numpy as np
import json
import tensorflow as tf
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

# Load the JSON configuration and vocab files
def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Load vocab and vectorization configurations
eng_vectorization_config = load_json('C:\\Users\\Jay\\Machine_Translation-main\\Machine_Translation-main\\eng_vectorization_config.json')
spa_vectorization_config = load_json('C:\\Users\\Jay\\Machine_Translation-main\\Machine_Translation-main\\spa_vectorization_config.json')
eng_vocab = load_json('C:\\Users\\Jay\\Machine_Translation-main\\Machine_Translation-main\\eng_vocab.json')
spa_vocab = load_json('C:\\Users\\Jay\\Machine_Translation-main\\Machine_Translation-main\\spa_vocab.json')

# Initialize tokenizers
eng_tokenizer = Tokenizer(num_words=vocab_size)
eng_tokenizer.word_index = {word: index for index, word in enumerate(eng_vocab)}

spa_tokenizer = Tokenizer(num_words=vocab_size)
spa_tokenizer.word_index = {word: index for index, word in enumerate(spa_vocab)}

# Load the trained model
transformer = load_model("transformer_model.keras", custom_objects={
    "TransformerEncoder": TransformerEncoder,
    "TransformerDecoder": TransformerDecoder,
    "PositionalEmbedding": PositionalEmbedding
})

# Function to vectorize input sentences
def vectorize_english(sentences, tokenizer, max_length):
    sequences = tokenizer.texts_to_sequences(sentences)
    return pad_sequences(sequences, maxlen=max_length, padding='post')


# Beam Search Decoder class
class BeamSearchDecoder:
    def __init__(self, model, tokenizer, beam_width, max_seq_len):
        self.model = model
        self.tokenizer = tokenizer
        self.beam_width = beam_width
        self.max_seq_len = max_seq_len
        self.start_token = tokenizer.word_index.get('[start]', 0)
        self.end_token = tokenizer.word_index.get('[end]', 0)
        self.vocab_size = len(tokenizer.word_index)

    def decode(self, input_seq):
        beam = [(input_seq, [self.start_token], 0)]
        for _ in range(self.max_seq_len):
            new_beam = []
            for input_seq, output_seq, score in beam:
                output_seq_padded = pad_sequences([output_seq], maxlen=self.max_seq_len, padding='post')
                predictions = self.model.predict([input_seq, output_seq_padded], verbose=0)
                top_k_indices = np.argsort(predictions[0, len(output_seq)-1, :])[-self.beam_width:]
                for index in top_k_indices:
                    new_score = score + np.log(predictions[0, len(output_seq)-1, index])
                    new_seq = output_seq + [index]
                    new_beam.append((input_seq, new_seq, new_score))
            beam = sorted(new_beam, key=lambda x: x[2], reverse=True)[:self.beam_width]
            if all(seq[-1] == self.end_token for _, seq, _ in beam):
                break
        return beam[0][1]

# Function to convert sequences to text
def sequence_to_text(sequence, tokenizer):
    index_to_word = {index: word for word, index in tokenizer.word_index.items()}
    return ' '.join([index_to_word.get(index, '') for index in sequence if index not in [0, tokenizer.word_index.get('[start]', -1), tokenizer.word_index.get('[end]', -1)]])

# Translation function using beam search
def translate_sentence(sentence, beam_width=3, max_seq_len=20):
    input_seq = vectorize_english([sentence], eng_tokenizer, eng_vectorization_config.get('max_len', 20))
    decoder = BeamSearchDecoder(transformer, spa_tokenizer, beam_width, max_seq_len)
    decoded_sequence = decoder.decode(input_seq)
    return sequence_to_text(decoded_sequence, spa_tokenizer)

# Test the beam search decoder on your model
test_sentences = [
    "I asked Tom to meet me at the train station",
    "The exhibition will be open for another month",
    "This is what I was waiting for",
    "Your father loves you",
    "It cost me 10 dollars"
]

for sentence in test_sentences:
    translated = translate_sentence(sentence)
    print(f"Input: {sentence}")
    print(f"Translated: {translated}")
    print()


Input: I asked Tom to meet me at the train station
Translated: yo ella que tom familia su fácil me [UNK]

Input: The exhibition will be open for another month
Translated: el amo es un amo de [UNK]

Input: This is what I was waiting for
Translated: 

Input: Your father loves you
Translated: hecho que están o

Input: It cost me 10 dollars
Translated: me corriendo el sonrió

